## Connessione asincrona

In [1]:
import time
import threading
from ib_async import *
util.startLoop()  # uncomment this line when in a notebook
import numpy as np

#-------------------------------------------------------
#MY INPUTS:
ib = IB()
id_connection = float(input('Enter the connection id: '))
lookback_window = int(input('Enter the lookback window: '))
number_sdev = int(input('Enter the number of standard deviations: '))
#-------------------------------------------------------



#-------------------------------------------------------
#FUNZIONI ASINCRONE:

def connect_ib():
    while not ib.isConnected():
        try:
            ib.connect('127.0.0.1', 4002, clientId=id_connection)
            print("Connected to IB")
        except Exception as e:
            print(f"Connection failed: {e}. Retrying in 60 seconds...")
            time.sleep(60)



# Function to resubscribe to data feed
def resubscribe_data():
    global bars
    contract = Forex('EURGBP')
    bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='900 S',
        barSizeSetting='5 secs',
        whatToShow='MIDPOINT',
        useRTH=False,
        formatDate=1,
        keepUpToDate=True)
    bars.updateEvent += onBarUpdate


#we define the restart function
def RestartFunction():
    resubscribe_data()
    ib.run()


def check_connection():
    while True:
        if not ib.isConnected():
            print("Disconnected from IB. Attempting to reconnect...")
            connect_ib()
            RestartFunction()

        time.sleep(60)  # Check every 60 seconds




#-------------------------------------------------------
#INIZIALIZZAZIONE e salvataggio dei dati di mercato:
#-------------------------------------------------------
connect_ib() #mi connetto all'exchange
contract = Forex('EURGBP')
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='900 S',
        barSizeSetting='5 secs',
        whatToShow='MIDPOINT',
        useRTH=False,
        formatDate=1,
        keepUpToDate=True)

close_prices = [] #crea una lista
datetime = []
positions_list = []
positions_list.append(0)


position_size_gbp = float(100000) #size in GBP



#-------------------------------------------------------
#LOGICA DELLA STRATEGIA:
#-------------------------------------------------------

def onBarUpdate(bars, hasNewBar):
    #logging features:
    close_prices.append(float(bars[-1].close)) #append il prezzo di chiusura alla lista
    datetime.append(str(bars[-1].date))
    ohlc = [] #crea una lista
    ohlc.append(str(bars[-1].date)) #append la data
    ohlc.append(str(bars[-1].close))

    position_size_eur = position_size_gbp / float(bars[-1].close) #calcola la size in EUR



    #calculate the bollinger bands:
    if len(close_prices) >= lookback_window:
        std = np.std(close_prices[-lookback_window:]) #calcola la deviazione standard dei 20 prezzi precedenti
        sma = np.mean(close_prices[-lookback_window:])
        upper_band = sma + number_sdev * std #calcola la banda superiore
        lower_band = sma - number_sdev * std #calcola la banda inferiore
        #ohlc.append(str(upper_band)) #not necessary, just for testing
        #ohlc.append(str(lower_band)) #same here...
        ohlc.append(str(sma))
    else:
        ohlc.append('N/A')  # Not enough data for Bollinger Bands
        positions_list.append(0) #we continue to append 0 to the positions list since we don't have enough data to make a decision
    
    #--------------------------------------------------------------------
    
    if len(close_prices) >= lookback_window:
        if positions_list[-1] == 0:
            # ib is not in a position
            if float(bars[-1].close) > float(upper_band):
                positions_list.append(-1)
                ib.placeOrder(contract, MarketOrder('SELL', totalQuantity=position_size_gbp))
            elif float(bars[-1].close) < float(lower_band):
                positions_list.append(1)
                ib.placeOrder(contract, MarketOrder('BUY', totalQuantity=position_size_eur))
        
        else:
            # ib is in a position
            if positions_list[-1] == 1 and float(bars[-1].close) > float(sma):
                positions_list.append(0)
                ib.placeOrder(contract, MarketOrder('SELL', totalQuantity=position_size_gbp))
            elif positions_list[-1] == -1 and float(bars[-1].close) < float(sma):
                positions_list.append(0)
                ib.placeOrder(contract, MarketOrder('BUY', totalQuantity=position_size_eur))
    #--------------------------------------------------------------------

    print(",".join(ohlc), 'current position is:', positions_list[-1]) #printa la lista separata da virgole


# Run the connection checker in a separate thread
import threading
connection_thread = threading.Thread(target=check_connection)
connection_thread.start()

bars.updateEvent += onBarUpdate
ib.run()

Connected to IB


2024-09-13 06:46:10-04:00,0.8446,N/A current position is: 0


Value ([BarData(date=datetime.datetime(2024, 9, 13, 6, 31, 10, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.8447, high=0.84475, low=0.8447, close=0.84475, volume=-1.0, average=-1.0, barCount=-1), BarData(date=datetime.datetime(2024, 9, 13, 6, 31, 15, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.84475, high=0.84475, low=0.8447, close=0.8447, volume=-1.0, average=-1.0, barCount=-1), BarData(date=datetime.datetime(2024, 9, 13, 6, 31, 20, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.8447, high=0.8447, low=0.8447, close=0.8447, volume=-1.0, average=-1.0, barCount=-1), BarData(date=datetime.datetime(2024, 9, 13, 6, 31, 25, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.8447, high=0.84475, low=0.8447, close=0.84475, volume=-1.0, average=-1.0, barCount=-1), BarData(date=datetime.datetime(2024, 9, 13, 6, 31, 30, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.84475, high=0.84475, low=0.8447, close=0.84475, volume=-1.0, average=-1.0, barCount=-1), BarData(date=datet

2024-09-13 06:46:15-04:00,0.84465,N/A current position is: 0
2024-09-13 06:46:20-04:00,0.84465,N/A current position is: 0
2024-09-13 06:46:25-04:00,0.84465,N/A current position is: 0
2024-09-13 06:46:30-04:00,0.84465,N/A current position is: 0
2024-09-13 06:46:35-04:00,0.8447,N/A current position is: 0
2024-09-13 06:46:35-04:00,0.84465,N/A current position is: 0
2024-09-13 06:46:40-04:00,0.84465,N/A current position is: 0
2024-09-13 06:46:40-04:00,0.84465,N/A current position is: 0
2024-09-13 06:46:45-04:00,0.84465,N/A current position is: 0
2024-09-13 06:46:45-04:00,0.8446,N/A current position is: 0
2024-09-13 06:46:50-04:00,0.84465,N/A current position is: 0
2024-09-13 06:46:50-04:00,0.84465,N/A current position is: 0
2024-09-13 06:46:55-04:00,0.84465,N/A current position is: 0
2024-09-13 06:46:55-04:00,0.84465,N/A current position is: 0
2024-09-13 06:47:00-04:00,0.84465,N/A current position is: 0
2024-09-13 06:47:05-04:00,0.84465,N/A current position is: 0
2024-09-13 06:47:05-04:00,

Value ([BarData(date=datetime.datetime(2024, 9, 13, 6, 31, 10, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.8447, high=0.84475, low=0.8447, close=0.84475, volume=-1.0, average=-1.0, barCount=-1), BarData(date=datetime.datetime(2024, 9, 13, 6, 31, 15, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.84475, high=0.84475, low=0.8447, close=0.8447, volume=-1.0, average=-1.0, barCount=-1), BarData(date=datetime.datetime(2024, 9, 13, 6, 31, 20, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.8447, high=0.8447, low=0.8447, close=0.8447, volume=-1.0, average=-1.0, barCount=-1), BarData(date=datetime.datetime(2024, 9, 13, 6, 31, 25, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.8447, high=0.84475, low=0.8447, close=0.84475, volume=-1.0, average=-1.0, barCount=-1), BarData(date=datetime.datetime(2024, 9, 13, 6, 31, 30, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=0.84475, high=0.84475, low=0.8447, close=0.84475, volume=-1.0, average=-1.0, barCount=-1), BarData(date=datet

Disconnected from IB. Attempting to reconnect...
Connected to IB


Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 10182, reqId 631: Failed to request live updates (disconnected).
Peer closed connection.


### Test vari

In [ ]:
import ib_async, threading
import time
from ib_async import *
util.startLoop()  # uncomment this line when in a notebook

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=2)
if ib.isConnected():
    print("Connected to IB")

ib.run()

In [ ]:
ib.disconnect()